In [ ]:
import pandas as pd
import numpy as np
import re #para identificar patrones en textos
from langdetect import detect #para detectar idiomas, se tiene que instalar
from deep_translator import GoogleTranslator #para traducir texto, se tiene que instalar

## Description

In [ ]:
def treat_euro(text):
    text = re.sub(r'(euro[^s])|(euros)|(€)', ' euros', text)
    return text

def treat_m2(text):
    text = re.sub(r'(m2)|(m²)', ' m²', text)
    return text

def filter_ibans(text):
    pattern = r'ES\d{2}[ ]\d{4}[ ]\d{4}[ ]\d{4}[ ]\d{4}[ ]\d{2}|ES\d{20}|ES[ ]\d{2}[ ]\d{3}[ ]\d{3}[ ]\d{3}[ ]\d{5}'
    text = re.sub(pattern, '', text)
    return text

def remove_space_between_numbers(text):
    text = re.sub(r'(\d)\s+(\d)', r'\1\2', text)
    return text

def filter_emails(text):
    pattern = r'(?:(?!.*?[.]{2})[a-zA-Z0-9](?:[a-zA-Z0-9.+!%-]{1,64}|)|\"[a-zA-Z0-9.+!% -]{1,64}\")@[a-zA-Z0-9][a-zA-Z0-9.-]+(.[a-z]{2,}|.[0-9]{1,})'
    text = re.sub(pattern, '', text)
    return text

def filter_ref(text):
    pattern = r'(\(*)(ref|REF)(\.|[ ])\d+(\)*)'
    text = re.sub(pattern, '', text)
    return text

def filter_websites(text):
    pattern = r'(http\:\/\/|https\:\/\/)?([a-z0-9][a-z0-9\-]*\.)+[a-z][a-z\-]*'
    text = re.sub(pattern, '', text)
    return text

def filter_phone_numbers(text):
    pattern = r'(?:(?:\+|00)33[\s.-]{0,3}(?:\(0\)[\s.-]{0,3})?|0)[1-9](?:(?:[\s.-]?\d{2}){4}|\d{2}(?:[\s.-]?\d{3}){2})|(\d{2}[ ]\d{2}[ ]\d{3}[ ]\d{3})'
    text = re.sub(pattern, '', text)
    return text

def remove_large_numbers(text):
    pattern = r'\b\d{4,}\b'
    text = re.sub(pattern, '', text)
    return text

def remove_text_in_brackets(text):
    pattern = r'\[[^\]]*\]|\{[^}]*\}'
    text = re.sub(pattern, '', text)
    return text

def clean_text(text):
    if isinstance(text, str):  # Verifica si text es una cadena de texto
        text = text.replace(u'\xa0', u' ')
        text = treat_m2(text)
        text = treat_euro(text)
        text = filter_phone_numbers(text)
        text = filter_emails(text)
        text = filter_ibans(text)
        text = filter_ref(text)
        text = filter_websites(text)
        text = remove_space_between_numbers(text)
        text = remove_large_numbers(text)
        text = remove_text_in_brackets(text)
    return text

df['cl_descrip'] = df.Description.apply(clean_text)


# Reemplazar los valores NaN o no válidos con una cadena vacía ''
df['cl_descrip'] = df['cl_descrip'].fillna('')

# Función para detectar el idioma de un texto
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

# Agrega una nueva columna 'Language' con el idioma detectado para cada descripción
base['Language'] = base['cl_descrip'].apply(detect_language)

# Función para cortar el texto a un máximo de 300 palabras
def truncate_text(text):
    words = text.split()[:450]
    truncated_text = ' '.join(words)
    return truncated_text

# Aplico la función para truncar el texto en la columna 'cl_descrip'
base['cl_descrip'] = base['cl_descrip'].apply(truncate_text)

# Filtro las filas que tienen 'Language' algo distinto al español ('es')
no_esp_descriptions = base[base['Language'] != 'es']['cl_descrip']

# Traduzco las descripciones al español
translated_descriptions = []
for description in no_esp_descriptions:
    translated_description = GoogleTranslator(source='auto', target='es').translate(description)
    translated_descriptions.append(translated_description)

# Reemplazo las descripciones traducidas en el DataFrame original
base.loc[base['Language'] != 'es', 'cl_descrip'] = translated_descriptions

df=df.drop('Language', axis=1)

## 1. Price_other

In [48]:
# Crear DataFrame de ejemplo
datos = {'Price_other': [
    "255m2 - 4 habitaciones - 2 baños - 17,65€/m2",
    "150m2 - 3 habitaciones - 1 baño - 20,50€/m2",
    "200m2 - 5 habitaciones - 3 baños - 15,75€/m2"]}

df = pd.DataFrame(datos)
print(df)

                                    Price_other
0  255m2 - 4 habitaciones - 2 baños - 17,65€/m2
1   150m2 - 3 habitaciones - 1 baño - 20,50€/m2
2  200m2 - 5 habitaciones - 3 baños - 15,75€/m2


In [49]:
# Función para extraer la información del precio y otras características
def extraer_info(texto):
    area_match = re.search(r'(\d+)m2', texto)
    area = int(area_match.group(1)) if area_match else 0
    rooms_match = re.search(r'(\d+) habitaciones', texto)
    rooms = int(rooms_match.group(1)) if rooms_match else 0
    price_m2_match = re.search(r'(\d+,\d+)€/m2', texto)
    price_m2 = float(price_m2_match.group(1).replace(',', '.')) if price_m2_match else 0
    return pd.Series([area, rooms, price_m2])

# Aplicar la función y crear las nuevas columnas
df[['area', 'rooms', 'price_m2']] = df['Price_other'].apply(extraer_info)

df=df.drop('Price_other',axis=1)

print(df)

                                    Price_other   area  rooms  price_m2
0  255m2 - 4 habitaciones - 2 baños - 17,65€/m2  255.0    4.0     17.65
1   150m2 - 3 habitaciones - 1 baño - 20,50€/m2  150.0    3.0     20.50
2  200m2 - 5 habitaciones - 3 baños - 15,75€/m2  200.0    5.0     15.75


## 2. Last modification

In [ ]:
# Aplicar la función a la columna
df['last_mod'] = df['Last Modification'].apply(lambda x: x.replace('última modificación ', ''))
df=df.drop('Last Modification',axis=1)

## 3. Distribution

Primero reviso si no hay más información en esta columna que la de los índices de alquiler.

In [1]:
# Define la expresión regular para buscar el patrón de 'Índice Alquiler'
patron = r'Índice Alquiler:.*'

# Función para verificar si una cadena contiene el patrón
def verificar_estructura(cadena):
    return bool(re.search(patron, cadena))

# Aplica la función a la columna y crea una nueva columna 'estructura_similar'
df['estructura_similar'] = df['Distribution'].apply(verificar_estructura)

cantidad_false = df['estructura_similar'].value_counts().get(False, 0)
cantidad_false

                                             columna  estructura_similar
0  Información del precio\n\nPrecio alquiler: 4.5...                True
1  Información del precio\n\nPrecio alquiler: 3.2...                True
2  Información del precio\n\nPrecio alquiler: 2.8...               False


Si todos fueran `True`, podría usar el siguiente código para ver qué valores hay:

In [ ]:
# Función para extraer el texto después de 'Índice Alquiler:'
def extraer_texto_despues_indice_alquiler(cadena):
    match = re.search(patron, cadena)
    if match:
        return match.group(1).strip()  # Elimina espacios en blanco al inicio y al final
    else:
        return None

# Aplica la función a la columna y crea una nueva columna 'indice_alquiler'
df['indice_alquiler'] = df['Distribution'].apply(extraer_texto_despues_indice_alquiler)

# Muestra los valores únicos en toda la columna 'indice_alquiler'
valores_unicos = df['indice_alquiler'].unique()

print("Valores únicos en la columna 'indice_alquiler':")
print(valores_unicos)

## 4. General Characteristics

In [30]:
# Crear DataFrame de ejemplo
datos = {'General Characteristics': [
    'Distribución\n\n4 habitaciones\nSuperficie 255 m2\n2 Baños\nCocina tipo office: No',
    'Distribución\n\n2 habitaciones\nSuperficie 55 m2\n1 Baño',
    'Distribución\n\n3 habitaciones\nSuperficie 122 m2\n2 Baños\nCocina tipo office']}

df = pd.DataFrame(datos)

In [31]:
# Define expresiones regulares para extraer la información
patron_banos = r'(\d+) Baño'
patron_cocina_office = r'Cocina tipo office(: *(No|Si|Sí))?'

# Función para extraer el número de baños
def extraer_banos(texto):
    resultado = re.search(patron_banos, texto)
    if resultado:
        return int(resultado.group(1))
    else:
        return None

# Función para determinar si hay cocina tipo office
def cocina_office(texto):
    resultado = re.search(patron_cocina_office, texto)
    if resultado:
        return 1 if resultado.group(2) is not None else 0
    else:
        return 0

# Función para extraer la información adicional después de los baños
def extraer_informacion_adicional(texto):
    # Buscar la posición de la línea que contiene información de baños
    indice_banos = texto.find(' Baño')
    if indice_banos != -1:
        # Buscar si hay información sobre la cocina después de los baños
        info_cocina = re.search(patron_cocina_office, texto[indice_banos:])
        if info_cocina:
            # Si hay información sobre la cocina, extraer lo que viene después de ella
            return texto[indice_banos + info_cocina.end():]
        else:
            # Si no hay información sobre la cocina, extraer todo lo que viene después de los baños
            return texto[indice_banos + 6:]
    else:
        return None

# Aplicar las funciones y crear las nuevas columnas
df['baths'] = df['General Characteristics'].apply(extraer_banos)
df['kitchen_office'] = df['General Characteristics'].apply(cocina_office)
df['gc_ad'] = df['General Characteristics'].apply(extraer_informacion_adicional)

# Función para extraer la información de las características generales
def extraer_caracteristicas_generales(texto):
    terrace_match = re.search(r'Terraza (\d+) m²', texto)
    terrace = int(terrace_match.group(1)) if terrace_match else 0
    aseo_match = re.search(r'(\d+) (Aseo|Aseos)', texto)
    aseo = int(aseo_match.group(1)) if aseo_match else 0
    laundry = 1 if re.search(r'Lavadero', texto) else 0
    return pd.Series([terrace, aseo, laundry])

# Aplicar la función y crear las nuevas columnas
df[['terrace', 'aseo', 'laundry']] = df['General Characteristics'].apply(extraer_caracteristicas_generales)

# Eliminar las líneas adicionales de la columna original
df=df.drop('General Characteristics',axis=1)

print(df)

   Baths  Kitchen_office GC_ad
0      2               1      
1      1               0      
2      2               0      


Ahora quiero ver qué valores adicionales de distribución hay.

In [32]:
df['gc_ad'].unique()

array([''], dtype=object)

## 5. EPC Consum

In [67]:
import pandas as pd
import re

# Crear DataFrame de ejemplo
datos = {'EPC_Consum': [
    'Consumo:\n\n \n\n153 kW h m2 / año',
    'Consumo:\n\nD\n\n150 kW h m2 / año',
    'Consumo:\n\nG\n\n266 kW h m2 / año']}

df = pd.DataFrame(datos)
df

,EPC_Consum
0,Consumo:\n\n \n\n153 kW h m2 / año
1,Consumo:\n\nD\n\n150 kW h m2 / año
2,Consumo:\n\nG\n\n266 kW h m2 / año


In [68]:
# Función para extraer la letra de la segunda línea
def extraer_letra(texto):
    resultado = re.search(r'(?<=\n)\w(?=\n)', texto)
    if resultado:
        return resultado.group()
    else:
        return None

# Función para extraer el valor numérico de la tercera línea
def extraer_valor(texto):
    resultado = re.search(r'(?<=\n)\d+(?= kW h m2 / año)', texto)
    if resultado:
        return int(resultado.group())
    else:
        return None

# Aplicar las funciones y crear las nuevas columnas
df['consum_EPC'] = df['EPC_Consum'].apply(extraer_letra)
df['kwhm2_year'] = df['EPC_Consum'].apply(extraer_valor)

# Eliminar las líneas adicionales de la columna original
df= df.drop('EPC_Consum', axis=1)

print(df)

  consum_EPC  kwhm2_year
0       None         153
1          D         150
2          G         266


## 6. EPC Emission

In [42]:
import pandas as pd
import re

# Crear DataFrame de ejemplo
datos = {'EPC_Emission': [
    'Emisión:\n\nE\n\n153 kg CO2 m2 / año',
    'Emisión:\n\nD\n\n150 kg CO2 m2 / año',
    'Emisión:\n\nG\n\n266 kg CO2 m2 / año']}

df = pd.DataFrame(datos)
df

,EPC_Emission
0,Emisión:\n\nE\n\n153 kg CO2 m2 / año
1,Emisión:\n\nD\n\n150 kg CO2 m2 / año
2,Emisión:\n\nG\n\n266 kg CO2 m2 / año


In [43]:
# Función para extraer la letra de la segunda línea
def extraer_letra(texto):
    resultado = re.search(r'(?<=\n)\w(?=\n)', texto)
    if resultado:
        return resultado.group()
    else:
        return None

# Función para extraer el valor numérico de la tercera línea
def extraer_valor(texto):
    resultado = re.search(r'(?<=\n)\d+(?= kg CO2 m2 / año)', texto)
    if resultado:
        return int(resultado.group())
    else:
        return None

# Aplicar las funciones y crear las nuevas columnas
df['emission_EPC'] = df['EPC_Emission'].apply(extraer_letra)
df['kgco2m2_year'] = df['EPC_Emission'].apply(extraer_valor)

# Eliminar las líneas adicionales de la columna original
df= df.drop('EPC_Emission', axis=1)

print(df)

  Emission_EPC  kgCO2m2_year
0            E           153
1            D           150
2            G           266


## 7. Equipment

In [46]:
import pandas as pd
import re

# Crear DataFrame de ejemplo
datos = {'Equipment': [
    "Características generales\n\nCalefacción \nSin plaza parking\nSin amueblar\nAire acondicionado \nAño construcción 1931\n\nCertificado energético : \n\n\nConsumo:\n\nE\n\n153  kW h m2 / año\nEmisiones: E\n32  kg CO2 m2 / año\n\nVer etiqueta calificación energética",
    "Características generales\n\nAire acondicionado \nPlanta número 3\nAño construcción 1900\n\nCertificado energético : \n\n\nConsumo:\n\nD\n\n150  kW h m2 / año\nEmisiones: D\n50  kg CO2 m2 / año\n\nVer etiqueta calificación energética",
    "Características generales\n\nSin plaza parking\nSin amueblar\nAire acondicionado \nAño construcción 1945\n\nCertificado energético : \n\n\nConsumo:\n\nG\n\n266  kW h m2 / año\nEmisiones: G\n41  kg CO2 m2 / año\n\nVer etiqueta calificación energética"]}

df = pd.DataFrame(datos)
print(df)

                                           Equipment
0  Características generales\n\nCalefacción \nSin...
1  Características generales\n\nAire acondicionad...
2  Características generales\n\nSin plaza parking...


In [47]:
# Función para extraer la información de las características generales
def extraer_caracteristicas(texto):
    ac = 1 if re.search(r'Aire acondicionado(\s*:\s*Sí)?', texto) else 0
    parking = 1 if re.search(r'Plaza parking', texto) else 0
    pool = 1 if re.search(r'Piscina comunitaria', texto) else 0
    lift = 1 if re.search(r'Ascensor', texto) else 0
    furniture = 1 if re.search(r'Amueblado', texto) else 0
    public_transp = 1 if re.search(r'Cerca de transporte público', texto) else 0
    year_construc_match = re.search(r'Año construcción (\d{4})', texto)
    year_construc = int(year_construc_match.group(1)) if year_construc_match else 0
    return pd.Series([ac, parking, pool, lift, furniture, public_transp, year_construc])

# Aplicar la función y crear las nuevas columnas
df[['AC', 'parking', 'pool', 'lift', 'furniture', 'public_transp', 'year_cons']] = df['Equipment'].apply(extraer_caracteristicas)

   AC  Parking  Pool  Lift  Furniture  Public_Transp  Year_Construc
0   1        0     0     0          0              0           1931
1   1        0     0     0          0              0           1900
2   1        0     0     0          0              0           1945


## 8. Floor

Le di esta indicación a ChatGPT:
En un dataframe con varias columnas quiero crear una nueva siguiendo estas reglas:
1. Si en la columna "Equipment" hay un texto como "Planta número", en la nueva columna (a la que llamaremos 'floor') indicar el valor numérico que tiene a continuación.
2. Si no se cumple la orden 1, revisar todas las demás columnas del dataframe y si en alguna menciona algo como "Planta baja", en la columna 'floor' indicar 0.
3. Si no se cumple la regla 1 ni 2, en la columna 'floor' indicar el valor 999.

In [62]:
import pandas as pd
import re

# Ejemplo de dataframe
data = {
    'Equipment': ['Planta número 3', 'Planta 5', 'Planta número 8', '5 baños'],
    'Other_Column': ['Algo', 'Otro', 'Planta baja', 'Planta baja']
}

df = pd.DataFrame(data)
print(df)

         Equipment Other_Column
0  Planta número 3         Algo
1         Planta 5         Otro
2  Planta número 8  Planta baja
3          5 baños  Planta baja


In [63]:
# Función para aplicar las reglas y crear la nueva columna 'floor'
def assign_floor(row):
    if re.search(r'Planta número (\d+)', row['Equipment']):
        return int(re.search(r'Planta número (\d+)', row['Equipment']).group(1))
    elif re.search(r'Planta baja', ' '.join(row)):
        return 0
    else:
        return 999

# Aplicar la función a cada fila del dataframe
df['floor'] = df.apply(assign_floor, axis=1)

# Eliminar la columna original
df = df.drop('Equipment',axis=1)

         Equipment Other_Column  floor
0  Planta número 3         Algo      3
1         Planta 5         Otro    999
2  Planta número 8  Planta baja      8
3          5 baños  Planta baja      0


## 9. Adicionales

In [ ]:
# Definir función para extraer el valor numérico
def extraer_valor_numerico(texto):
    resultado = re.search(r'\b(\d+(?:\.\d+)?)\b', texto)
    if resultado:
        return float(resultado.group(1))
    else:
        return None

# Aplicar la función y crear la nueva columna 'precio_num'
df['precio_eur'] = df['precio_eur'].apply(extraer_valor_numerico)

In [ ]:
# Dividir el texto por el guión y seleccionar la segunda parte
df['location'] = df['Location'].str.split(' - ', expand=True)[1]

df=df.drop('Location',axis=1)

In [ ]:
# Definir función para extraer los valores de longitud y latitud
def extraer_lon_lat(texto):
    lon = re.search(r'(\d+\.\d+)', texto).group(1)
    lat = re.search(r'(?<=VGPSLon\":-)(\d+\.\d+)', texto).group(1)
    return pd.Series([lon, lat])

# Aplicar la función y crear las nuevas columnas 'Lon_X' y 'Lat_Y'
df[['Lon_X', 'Lat_Y']] = df['Lon/Lat'].apply(extraer_lon_lat)

df=df.drop('Lon/Lat',axis=1)

## 10. Revisión final

In [ ]:
print(df.columns.tolist())

In [ ]:
df.head(10)